### 텐서

파이토치의 텐서는 넘파이의 배열인 **ndarray**와 같은 개념입니다. 추가로 텐서간 연산에 따른 그래와 $경사도^{gradient}$를 저장할 수 있습니다. 따라서 파이토치에서 연산을 수행하기 위한 가장 기본적인 객체로써 모든 연산은 이 객체를 통하게 됩니다. 파이토치는 텐서를 통해 값을 저장하고 그 값들에 대한 연산을 수행할 수 있는 함수를 제공합니다.

#### 파이토치 코드와 넘파이 코드 비교

In [1]:
import torch
import numpy as np

x = torch.Tensor([[1,2,], [3, 4]])
y = np.array([[1, 2], [3, 4]])
z = torch.from_numpy(y)

두 코드 모두 다음과 같은 x라는 변수에 2x2 $행렬^{matrix}$을 만들어냅니다.

<br></br>
![](./images/3-3-1-matrix.jpg)
<br></br>

이처럼 파이토치는 넘파이와 비슷한 방식의 코딩 스타일을 가지며, 따라서 코드를 보고 해석하거나 새롭게 작성할때 매우 수월합니다. 또한 파이토치는 아래와 같은 데이터 타입을 파이토치 텐서로 제공합니다.

<br></br>
![](./images/3-3-1-dtype.jpg)
<br></br>

### Autograd

파이토치는 자동으로 미분및 역전파를 수행하는 **autograd** 기능을 가집니다. 대부분의 경우 텐서간의 연산들을 크게 신경 쓸 필요 없이 역전파 알고리즘을 수행하는 명령어를 호출하기만 하면 됩니다.

파이토치는 텐서들간에 연산을 수행할때마다 동적으로 $연산 그래프^{computation graph}$를 생성하여 연산의 결과물이 어떤 텐서로부터 어떤 연산을 통해서 왔는지 추적합니다. 따라서 우리가 최종적으로 나온 $스칼라^{scalar}$에 역전파 알고리즘을 통해 미분을 수행하도록 했을때 각 텐서는 자기 자신의 자식 노드에 해당하는 텐서와 연산을 자동으로 찾아 계속해서 역전파 알고리즘을 수행할 수 있도록 합니다.

In [2]:
import torch

x = torch.FloatTensor(2, 2)
y = torch.FloatTensor(2, 2)
y.requires_grad_(True)

z = (x + y) + torch.FloatTensor(2, 2)

이 예제에서처럼 x와 y를 생성하고 둘을 더하는 연산을 수행하면 x + y에 해당하는 텐서가 생성되어 연산 그래프에 할당됩니다. 그리고 다시 생성된 텐서를 더해준 뒤, 이를 z에 할당합니다. 따라서 z로부터 역전파를 수행하면 이미 생성된 연산 그래프에 따라서 미분 값을 전달할 수 있습니다.

<br></br>
![](./images/3-3-1-autograd.jpg)
<br></br>

이것이 기존 케라스, 텐서플로우와의 다른 점입니다. 케라스와 텐서플로우는 미리 정의한 연산들을 컴파일을 통해 고정한 후, 정해진 입력에 맞춰 텐서를 피드포워드해야 합니다. 반면 파이토치에는 **정해진 연산**이라는 것이 없고 모델은 *배워야하는 파라미터 텐서*만 알고있을뿐 그 **$가중치 파라미터^{weight parameter}$**들이 어떠한 연산을 통해 학습 또는 연산에 관여하는지 알 수 없습니다. 연산이 수행된 **직후**에 알 수 있을뿐입니다.

기울기를 구할 필요가 없는 연산의 경우에는 다음과 같이 **with** 문법을 사용하여 연산을 수행할 수 있습니다. 역전파 알고리즘 수햄이 필요없는 비학습 과정, 즉 예측, 추론 등을 수행할때 유용하며, 기울기를 구하기위한 사전 작업을 생략할 수 있으므로 연산 속도 및 메모리 사용 측면에서도 큰 이점이 있습니다.

In [3]:
import torch

x = torch.FloatTensor(2, 2)
y = torch.FloatTensor(2, 2)
y.requires_grad_(True)

with torch.no_grad():
    z = (x + y) + torch.FloatTensor(2, 2)

### 피드포워드

이번에는 선형 계층 **linear layer** (또는 완전연결계층 **fully-connected layer**)를 구현해보겠습니다.

M x N의 입력 행렬 x가 주어지면, N x P의 행렬 W를 곱한 후 P차원의 벡터 b를 편차값으로 더합니다.

$y = xW + b$

where $x \in R^{MxN}, W \in R^{NxP}, b \in R^P$

이 수식에서 x는 벡터이지만, 보통 딥러닝을 수행할때는 **미니배치**기준으로 수행하므로 x가 2차원 행렬이라고 가정할 수 있습니다. 좀 더 구현하기 쉽게 다음과 같이 표현할 수 있습니다.

$y = f(x;\theta)$

where $\theta = \{W, B\}$

이러한 선형 계층의 기능은 다음과 같이 파이토치로 구현할 수 있습니다.

In [4]:
import torch

def linear(x, W, b):
    y = torch.mm(x, W) + b
    
    return y

x = torch.FloatTensor(16, 10)
W = torch.FloatTensor(10, 5)
b = torch.FloatTensor(5)

y = linear(x, W, b)

y

tensor([[        inf, -2.7659e+11,         inf,  1.7089e-02,  5.3336e+17],
        [        inf,  1.6261e-03,  6.6041e+15,  5.4965e-04,         inf],
        [        inf,        -inf,         inf,         inf,         inf],
        [        inf,         nan,         nan,         inf,         inf],
        [        inf,  1.8149e+05,         nan,         nan,         inf],
        [        inf,        -inf,         inf,        -inf,         inf],
        [        inf,        -inf,         nan,         inf,         inf],
        [        inf,         inf,         nan,         inf,         inf],
        [        nan,        -inf,         nan,        -inf,         inf],
        [        inf,         nan,         nan,         inf,         inf],
        [        inf,  3.2923e+09,         nan,         inf,         inf],
        [        nan,        -inf,         nan,        -inf, -8.7206e+16],
        [       -inf,         inf,        -inf,         inf,         nan],
        [        inf,    

### nn.Module

이번에는 수식을 더 편리하고 깔끔하게 구현하는 방법을 살펴보겠습니다.

파이토치는 `nn.Module`이라는 클래스를 제공하여 사용자가 그 위에서 필요한 모델 구조를 표현할 수 있습니다.

`nn.Module`을 상속한 사용자 정의 클래스는 다시 내부에 `nn.Module`을 상속한 클래스 객체를 선언하여 소유할 수 있습니다. 즉 `nn.Module` 상속 개체 안에 `nn.Module` 상속 객체를 선언하여 변수로 사용할 수 있습니다. 그리고 `nn.Module`의 `forward()` 함수를 $오버라이드^{override}$하여 피드포워드를 구현할 수 있습니다.

이외에도 `nn.Module`의 특징을 이용하여 한번에 신경망 가중치 파라미터들을 저장 및 불러오기할 수 있습니다.

앞서 구현한 `linear` 함수 대신 `MyLinear`라는 클래스를 `nn.Module`을 상속받아 선언하고 이를 사용하여 똑같은 기능을 구현해보겠습니다.

In [5]:
import torch
import torch.nn as nn

class MyLinear(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.W = torch.FloatTensor(input_size, output_size)
        self.b = torch.FloatTensor(output_size)
        
    def forward(self, x):
        y = torch.mm(x, self.W) + self.b
        
        return y

In [6]:
x = torch.FloatTensor(16,10)
linear = MyLinear(10,5)
y = linear(x)

y

tensor([[2.2153e-07, 7.6471e-19, 2.2153e-07, 1.1113e-10, 2.7023e-14],
        [       inf,        inf,        inf, 1.2292e+11,        inf],
        [8.4307e+29, 4.7919e+16,        inf,        inf, 1.1827e+31],
        [1.6640e+09, 2.2346e-18, 1.1960e+14, 3.0263e+16, 7.3590e+12],
        [2.9758e+16, 3.0970e-41, 2.9758e+16, 3.0970e-41, 2.7022e-14],
        [7.0729e+09, 6.4827e+00, 2.8165e-01, 4.1334e-27, 2.7023e-14],
        [1.0048e+02, 2.5298e-28, 1.1077e-07, 8.3049e-23, 4.5938e-13],
        [3.0164e+19, 6.5714e+03, 3.0164e+19, 1.1113e-10, 2.7023e-14],
        [3.0164e+19, 6.5714e+03, 3.0164e+19, 8.7240e-23, 7.3589e+12],
        [2.6328e-14, 2.8551e-21, 3.7472e-14, 1.1113e-10, 2.8194e-21],
        [       inf, 8.2733e+24, 2.1926e+30, 2.7160e+30, 3.1159e+16],
        [2.0424e+11, 1.8720e+02, 8.1330e+00, 2.2226e-10, 5.4045e-14],
        [7.2508e+32, 2.4130e-23, 7.2508e+32, 3.0970e-41, 5.4045e-14],
        [7.1699e+12, 6.5715e+03, 7.1699e+12, 3.0264e+16, 2.6329e-18],
        [7.0729e+09,

`forward()`에서 정의한 대로 잘 동작하는 것을 볼 수 있습니다. 하지만 이처럼 W와 b를 선언하면 문제가 있습니다. `parameters()`함수는 모듈 내에서 선언된 학습이 필요한 파라미터들을 반환하는 $이터레이터^{iterator}$입니다. `linear` 모듈 내 학습이 핋요한 파라미터들의 크기를 `size()` 함수를 통해 확인해보겠습니다.

In [7]:
params = [p.size() for p in linear.parameters()]
print(params)

[]


아무것도 들어있지 않은 빈 리스트가 찍혀있습니다. 즉, `linear` 모듈 내에는 학습 가능한 파라미터가 없다는 이야기입니다. 신경망의 학습 파라미터는 단순한 텐서가 아니기 때문에 파라미터로 등록되어야 합니다. 따라서 우리는 `Parameter`라는 클래스를 사용하여 텐서를 감싸야합니다.

In [8]:
class MyLinear(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()
        
        self.W = nn.Parameter(torch.FloatTensor(input_size, output_size), requires_grad = True)
        self.b = nn.Parameter(torch.FloatTensor(output_size), requires_grad = True)
        
    def forward(self, x):
        y = torch.mm(x, self.W) + self.b
        
        return y       

그럼 다시 `linear` 모듈 내부의 학습 가능한 파라미터들의 크기를 확인하겠습니다.

In [9]:
linear = MyLinear(10, 5)

params = [p.size() for p in linear.parameters()]
print(params)

[torch.Size([10, 5]), torch.Size([5])]


잘 들어있는 것을 확인할 수 있습니다. 다시 깔끔하게 바꾸어 봅니다.

In [10]:
class MyLinear(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(MyLinear, self).__init__()
        
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        y = self.linear(x)
        
        return y

앞에서 `nn.Module`을 상속받은 클래스는 `nn.Module`의 자식 클래스를 멤버 변수로 가질 수 있다고 했습니다. 따라서 `nn.Linear` 클래스를 사용하여 `W`와 `b`를 대체했습니다. 그리고 다음과 같이 출력해보면 내부의 가중치 파라미터가 잘 출력되는 것을 확인할 수 있습니다.

In [11]:
linear = MyLinear(10, 5)
print(linear)

MyLinear(
  (linear): Linear(in_features=10, out_features=5, bias=True)
)


### 역전파 수행

지금까지 원하는 연산을 통해 값을 앞으로 전달하는 *피드포워드* 방법을 살펴보았습니다. 이제 이렇게 피드포워드를 통해 얻은 값에서 실제 정답값과의 차이를 계산하여 오류(손실)를 뒤로 전달하는 $역전파^{backpropagation}$ 알고리즘을 수행해보겠습니다.

예를 들어 우리가 원하는 값이 100이라고 했을때 `linear`의 결과값 텐서의 합과 목표값과의 거리 (error 또는 loss)를 구하고, 그 값에 대해 `backward()`함수를 사용하여 기울기를 구합니다. 이때 loss값은 **스칼라**로 표현되어야합니다.

In [12]:
objective = 100

x = torch.FloatTensor(16, 10)
linear = MyLinear(10, 5)
y = linear(x)

loss = (objective - y.sum())

loss.backward()

$x \in R^{16x10}, \theta = \{w,b\} and W \in R^{10x5}, b \in R^5 \\
\hat{y} = x \dot W + b \\
\hat{y} \in R^{16 x 5}$

<br></br>

이와 같이 구한 각 파라미터의 기울기에 대해서 반복적으로 경사하강법을 사용하여 에러를 줄여나갈 수 있습니다.

### train(), eval()

```python
# Training
linear.eval()

# Do some inference process
linear.train()
```
<br></br>

이처럼 파이토치가 제공하는 `train()`과 `eval()` 함수를 활용하면, 사용자는 필요에 따라 모델에 대해 *훈련 시*와 *추론 시*의 모드를 쉽게 전환할 수 있습니다. `nn.Module`을 상속받아 구현하고 생성한 객체는 기본적으로 훈련 모드입니다. 이를 `eval()`을 사용하여 추론 모드로 바꿔주면 *드롭아웃* 또는 *배치 정규화*와 같은 학습과 추론시 서로 다른 `forward()` 동작을 하는 모듈들에 대해서도 각 상황에 따라 올바르게 동작합니다. 다만 추론이 끝나면 다시 `train()`을 선언하여 원래의 훈련 모드로 돌아가야 합니다.

### 선형회귀분석 예제

이제까지 배운 것들을 활용하여 임의의 함수를 근사하는 회귀분석 예제를 구현해보겠습니다.

1. 임의로 생성한 텐서들을
2. 근사하고자 하는 정답 함수에 넣어 정답 (y)를 구하고
3. 그 정답과 신경망을 통과한 $hat{y}$과의 차이를 *평균 제곱 오차 (MSE)*를 통해 구하여
4. 확률적 경사하강법 **SGD**를 통해 최적화해보겠습니다.

MSE의 수식은 다음과 같습니다.
<br></br>
![](./images/3-3-7-MSE.jpg)
<br></br>

1. 먼저 1개의 선형 계층을 가진 `MyModel`이라는 모듈을 선언합니다.

In [13]:
import random

import torch
import torch.nn as nn

class MyModel(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(MyModel, self).__init__()
        
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        y = self.linear(x)
        
        return y

2. 그리고 다음과 같이 임의의 함수 `f` 가 동작한다고 가정합니다. 이때 함수 `f`가 내부적으로 어떻게 동작하는지 알고자 합니다. 그러면 손실함수를 최소로 만드는 파라미터 $\theta$를 찾아서 함수 `f`를 근사해야 합니다.

<br></br>
![](./images/3-3-7-MSE2.jpg)
<br></br>

이를 파이썬으로 구현하면 다음과 같습니다.

In [14]:
def ground_truth(x):
    return 3 * x[:,0] + x[:, 1] - 2 * x[:,2]

3. 다음은 모델과 텐서를 입력받아 *피드포워딩*한 후, 역전파 알고리즘을 수행하여 **경사하강법**의 한 스텝을 수행하는 함수힙니다.

In [15]:
def train(model, x, y, optim):
    # initialize gradients in all parameters in module
    optim.zero_grad()
    
    # feed-forward
    y_hat = model(x)
    
    # get error between answer and inferenced
    loss = ((y - y_hat)**2).sum() / x.size(0)
    
    # back-propagation
    loss.backward()
    
    # one-step of gradient descent
    optim.step()
    
    return loss.data

4. 앞의 함수들을 사용하기 위해 하이퍼파라미터를 설정하겠습니다.

In [16]:
batch_size = 1
n_epochs = 1000
n_iter = 10000

model = MyModel(3, 1)
optim = torch.optim.SGD(model.parameters(), lr = 1e-4, momentum = 0.1)

print(model)

MyModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)


5. 이 값을 사용하여 평균 손실값이 0.001보다 작아질때까지 훈련시킵니다.

In [17]:
for epoch in range(n_epochs):
    avg_loss = 0
    
    for i in range(n_iter):
        x = torch.rand(batch_size, 3)
        y = ground_truth(x.data)
        
        loss = train(model, x, y, optim)
        
        avg_loss += loss
        avg_loss = avg_loss / n_iter
        
    # simple test sample to check the network
    x_valid = torch.FloatTensor([[.3, .2, .1]])
    y_valid = ground_truth(x_valid.data)
    
    model.eval()
    y_hat = model(x_valid)
    model.train()
    
    print(avg_loss, y_valid.data[0], y_hat.data[0, 0])
    
    if avg_loss < 0.001:
        break

tensor(1.2396e-05) tensor(0.9000) tensor(0.8080)


이와 같이 임의의 함수에 대해 실제로 신경망을 근사하는 아주 간단한 예제를 살펴보았습니다. 내부에 계층이 1개뿐이고, 비선형 활성화 함수 *(non-linear activation function)*가 포함되지 않았으므로, 비선형적인 **신경망**이라기보다는 선형 *함수*라고 봐야합니다.

파이토치에서 딥러닝을 수행하는 과정을 다음과 같이 요약할 수 있습니다.

1. `nn.Module` 클래스를 상속받아 모델 아키텍쳐 클래스 선언
2. 해당 클래스 객체 생성
3. SGD나 Adam 등의 옵티마이저를 생성하고, 생성한 모델의 파라미터를 최적화 대상으로 등록
4. 데이터로 미니배치를 구성하여 피드포워드 연산 그래프 생성
5. 손실 함수를 통해 최종 결과값 *scalar*와 손실값 *loss* 계산
6. 손실에 대해서 `backward()` 호출 -> 연산 그래프 상의 텐서들의 기울기가 채워짐
7. 3번의 옵티마이저에서 `step()` 호출하여 경사하강법 1 스텝 수행
8. 4번으로 돌아가 수렴 조건이 만족할때까지 반복 수행

### GPU 사용하기

파이토치는 GPU에서 훈련하는 방법도 제공합니다. 다음과 같이 `cuda()` 함수를 통해서 원하는 객체를 GPU 메모리에 복사학더나 이동시킬수 있습니다.

In [18]:
# Note that tensor is declared in torch.cuda
x = torch.cuda.FloatTensor(16, 10)
linear = MyLinear(10, 5)

# .cuda() le module move to GPU memory
linear.cuda()
y = linear(x)

또한 `cpu()` 함수를 통해서 다시 PC의 메모리로 복사하거나 이동시킬 수 있습니다.